In [ ]:
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet_ic')
brown_ic = wordnet_ic.ic('ic-brown.dat')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


In [ ]:
rat = wn.synset('rat.n.01')
lion = wn.synset('lion.n.01')
print("Lin: %s" %rat.lin_similarity(rat, brown_ic))

Lin: 1.0


In [ ]:
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
 
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0
 
    # For each word in the first sentence
    for syn1 in synsets1:
      arr_simi_score = []
      # print('=========================================')
      # print(syn1)
      # print('----------------')
      for syn2 in synsets2:
        # print(syn2)
        try:
          simi_score = syn1.lin_similarity(syn2,brown_ic)
        except:
          simi_score=None
        # print(simi_score)?
        if simi_score is not None:
          arr_simi_score.append(simi_score)
      # print('----------------')
      # print(arr_simi_score)
      if(len(arr_simi_score) > 0):
        best = max(arr_simi_score)
        # print(best)
        score += best
        count += 1

    # Average the values
    # print('score: ', score)
    # print('count: ', count)
    score /= count
    return score

sentences = [
    "Dogs are awesome.",
    "Some gorgeous creatures are felines.",
    "Dolphins are swimming mammals.",
    "Cats are beautiful animals.",
]
 
focus_sentence = "Cats are beautiful animals."
 
for sentence in sentences:
    print ("Similarity(\"%s\", \"%s\") = %s" % (focus_sentence, sentence, sentence_similarity(focus_sentence, sentence)))
    print ("Similarity(\"%s\", \"%s\") = %s" % (sentence, focus_sentence, sentence_similarity(sentence, focus_sentence)))

 
# Similarity("Cats are beautiful animals.", "Dogs are awesome.") = 0.8580557505390018
# Similarity("Dogs are awesome.", "Cats are beautiful animals.") = 0.9293976965291484

# Similarity("Cats are beautiful animals.", "Some gorgeous creatures are felines.") = 0.9903819953304521
# Similarity("Some gorgeous creatures are felines.", "Cats are beautiful animals.") = 0.9903819953304521

# Similarity("Cats are beautiful animals.", "Dolphins are swimming mammals.") = 0.8971083780190382
# Similarity("Dolphins are swimming mammals.", "Cats are beautiful animals.") = 0.6292623560819454

# Similarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0
# Similarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0 

Similarity("Cats are beautiful animals.", "Dogs are awesome.") = 0.861934542665209
Similarity("Dogs are awesome.", "Cats are beautiful animals.") = 0.9384004921866986
Similarity("Cats are beautiful animals.", "Some gorgeous creatures are felines.") = 0.9928986752928762
Similarity("Some gorgeous creatures are felines.", "Cats are beautiful animals.") = 0.9928986752928762
Similarity("Cats are beautiful animals.", "Dolphins are swimming mammals.") = 0.9013249930561423
Similarity("Dolphins are swimming mammals.", "Cats are beautiful animals.") = 0.6278107429022008
Similarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0
Similarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0


In [ ]:
def symmetric_sentence_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_similarity(sentence1, sentence2) + sentence_similarity(sentence2, sentence1)) / 2 
 
for sentence in sentences:
    print ("SymmetricSimilarity(\"%s\", \"%s\") = %s" % (
        focus_sentence, sentence, symmetric_sentence_similarity(focus_sentence, sentence)))
    print ("SymmetricSimilarity(\"%s\", \"%s\") = %s" % (
        sentence, focus_sentence, symmetric_sentence_similarity(sentence, focus_sentence)))

# SymmetricSimilarity("Cats are beautiful animals.", "Dogs are awesome.") = 0.9001675174259538
# SymmetricSimilarity("Dogs are awesome.", "Cats are beautiful animals.") = 0.9001675174259538

# SymmetricSimilarity("Cats are beautiful animals.", "Some gorgeous creatures are felines.") = 0.9928986752928762
# SymmetricSimilarity("Some gorgeous creatures are felines.", "Cats are beautiful animals.") = 0.9928986752928762

# SymmetricSimilarity("Cats are beautiful animals.", "Dolphins are swimming mammals.") = 0.7645678679791716
# SymmetricSimilarity("Dolphins are swimming mammals.", "Cats are beautiful animals.") = 0.7645678679791716

# SymmetricSimilarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0
# SymmetricSimilarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0

SymmetricSimilarity("Cats are beautiful animals.", "Dogs are awesome.") = 0.9001675174259538
SymmetricSimilarity("Dogs are awesome.", "Cats are beautiful animals.") = 0.9001675174259538
SymmetricSimilarity("Cats are beautiful animals.", "Some gorgeous creatures are felines.") = 0.9928986752928762
SymmetricSimilarity("Some gorgeous creatures are felines.", "Cats are beautiful animals.") = 0.9928986752928762
SymmetricSimilarity("Cats are beautiful animals.", "Dolphins are swimming mammals.") = 0.7645678679791716
SymmetricSimilarity("Dolphins are swimming mammals.", "Cats are beautiful animals.") = 0.7645678679791716
SymmetricSimilarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0
SymmetricSimilarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0
